In [0]:
%run ../Workspace/Shared/config/config

In [0]:
import re
import pyspark.sql.functions as F
from collections import defaultdict


In [0]:
# Função para limpar as colunas de data
def limpa_data(coluna):
    return (
        F.when(F.col(coluna).isNull() |
            ~F.col(coluna).rlike(r'^\d{8}$') | 
            F.col(coluna).isin('00000000'), None)
        .otherwise(F.to_date(F.col(coluna).cast('string'), 'yyyyMMdd'))
    )

def pega_prefixo(table_name):
    return re.sub(r'\d+$', '', table_name)

In [0]:
caminho_bronze = f'abfss://bronze@storagedatanexus.dfs.core.windows.net/'
caminho_silver = f'abfss://silver@storagedatanexus.dfs.core.windows.net/'

colunas = {
    'Cnaes': ['cod_cnae', 'descricao','_rescued_data','data_ingestao','data_ingestao_timestamp','caminho_fonte'],
    'Empresas': [
        'cnpj', 'razao_social', 'cod_natureza_juridica',
        'cod_qualificacao', 'capital_social',
        'porte_empresa', 'ente_federativo','_rescued_data',
        'data_ingestao', 'data_ingestao_timestamp', 'caminho_fonte'
    ],
    'Estabelecimentos': [
        'cnpj', 'cnpj_ordem', 'cnpj_verificador',
        'identificador_mtz_fil', 'nome', 'st_cadastral',
        'dt_st_cadastral', 'cod_motivo', 'nm_cid_exterior',
        'cod_pais', 'dt_inicio', 'cod_cnaes', 'cod_cnaes2',
        'tp_logradouro', 'logradouro', 'numero', 'complemento',
        'bairro', 'cep', 'uf', 'cod_municipio', 'ddd1',
        'telefone1', 'ddd2', 'telefone2', 'ddd_fax', 'fax',
        'email', 'st_especial', 'dt_st_especial','_rescued_data',
        'data_ingestao',
        'data_ingestao_timestamp',
        'caminho_fonte'
    ],
    'Motivos': ['cod_motivo', 'descricao','_rescued_data',
                'data_ingestao',
                'data_ingestao_timestamp',
                'caminho_fonte'],
    'Municipios': ['cod_municipio', 'descricao','_rescued_data',
                'data_ingestao',
                'data_ingestao_timestamp',
                'caminho_fonte'],
    'Paises': ['cod_pais', 'descricao','_rescued_data',
                'data_ingestao',
                'data_ingestao_timestamp',
                'caminho_fonte'],
    'Qualificacoes': ['cod_qualificacao', 'descricao', '_rescued_data',
                'data_ingestao',
                'data_ingestao_timestamp',
                'caminho_fonte'],
    'Simples': [
        'cnpj_basico', 'opcao_simples', 'data_opcao_simples',
        'data_exclusao_simples', 'opcao_mei',
        'data_opcao_mei', 'data_exclusao_mei','_rescued_data',
        'data_ingestao',
        'data_ingestao_timestamp',
        'caminho_fonte'
    ],
    'Socios': [
        'cnpj_basico', 'identificador_socio',
        'nome_socio_razao_social', 'cpf_cnpj_socio',
        'qualificacao_socio', 'data_entrada_sociedade',
        'pais', 'cpf_representante_legal',
        'nome_representante_legal',
        'qualificacao_representante_legal', 'faixa_etaria','_rescued_data',
        'data_ingestao',
        'data_ingestao_timestamp',
        'caminho_fonte'
    ],
    'Naturezas': ['codigo_natureza_juridica', 'descricao_natureza_juridica','_rescued_data',
        'data_ingestao',
        'data_ingestao_timestamp',
        'caminho_fonte']
}

In [0]:
dominios = [
    p.name.rstrip('/')
    for p in dbutils.fs.ls(f"{caminho_bronze}cnpj/")
                           if p.isDir()
]

dfs = {}

for df in dominios:
    path = f"{caminho_bronze}cnpj/{df}/"
    dfs[df] = spark.read.format('delta').load(path)
    dfs[df] = dfs[df].toDF(*colunas[df])

In [0]:
dfs_socios = dfs['Socios']
dfs_empresas = dfs['Empresas']
dfs_simples = dfs['Simples']
dfs_estabelecimentos = dfs['Estabelecimentos']
dfs_naturezas = dfs['Naturezas']
dfs_qualificacoes = dfs['Qualificacoes']
dfs_cnaes = dfs['Cnaes']
dfs_municipios = dfs['Municipios']
dfs_motivos = dfs['Motivos']
dfs_paises = dfs['Paises']

In [0]:
for nome_base in dfs:
    dfs[nome_base] = dfs[nome_base].dropDuplicates().dropna(how='all')

In [0]:
dfs_socios = (
    dfs_socios
    .withColumn('data_entrada_sociedade', F.to_date(F.col('data_entrada_sociedade'), 'yyyyMMdd'))
    .withColumn('faixa_etaria', F.col('faixa_etaria').cast('int'))
)

In [0]:
dfs_empresas = dfs_empresas.withColumn('capital_social',F.regexp_replace(F.col('capital_social'), ',', '.').cast('float'))

In [0]:
dfs_simples =(
    dfs_simples
    .withColumnRenamed('cnpj_basico', 'cnpj')
    .withColumn('cnpj', F.col('cnpj').cast('string'))
    .withColumn('data_opcao_simples', limpa_data('data_opcao_simples'))
    .withColumn('data_exclusao_simples', limpa_data('data_exclusao_simples'))
    .withColumn('data_opcao_mei', limpa_data('data_opcao_mei'))
    .withColumn('data_exclusao_mei', limpa_data('data_exclusao_mei'))
)

In [0]:
dfs_estabelecimentos = (
    dfs_estabelecimentos
        .withColumn('dt_st_cadastral', limpa_data('dt_st_cadastral'))
        .withColumn('dt_inicio', limpa_data('dt_inicio'))
        .withColumn('dt_st_especial', limpa_data('dt_st_especial'))
        .withColumn('cnae_divisao', F.substring('cod_cnaes', 1, 2))
)

In [0]:
dfs_socios.write.format('delta').mode('overwrite').option('overwriteSchema', 'true').save(caminho_silver + 'cnpj/socios')
dfs_empresas.write.format('delta').mode('overwrite').option('overwriteSchema', 'true').save(caminho_silver + 'cnpj/empresas')
dfs_simples.write.format('delta').mode('overwrite').option('overwriteSchema', 'true').save(caminho_silver + 'cnpj/simples')
dfs_estabelecimentos.write.format('delta').mode('overwrite').option('overwriteSchema', 'true').save(caminho_silver + 'cnpj/estabelecimentos')
dfs_naturezas.write.format('delta').mode('overwrite').option('overwriteSchema', 'true').save(caminho_silver + 'cnpj/naturezas')
dfs_qualificacoes.write.format('delta').mode('overwrite').option('overwriteSchema', 'true').save(caminho_silver + 'cnpj/qualificacoes')
dfs_cnaes.write.format('delta').mode('overwrite').option('overwriteSchema', 'true').save(caminho_silver + 'cnpj/cnaes')
dfs_municipios.write.format('delta').mode('overwrite').option('overwriteSchema', 'true').save(caminho_silver + 'cnpj/municipios')
dfs_motivos.write.format('delta').mode('overwrite').option('overwriteSchema', 'true').save(caminho_silver + 'cnpj/motivos')
dfs_paises.write.format('delta').mode('overwrite').option('overwriteSchema', 'true').save(caminho_silver + 'cnpj/paises')